# Spurious Correlations
***
## Introduction
This is a **short** *exploratory data analysis (EDA)* and *visualisation* of two random datasets. Even from this brief analysis, it is apparent that there are underlying issues within the datasets that would need further analysis and cleansing before serious, meaningful insights **could** and/or **should** be inferred. For further discussion on this see [Statistical Disclaimer Section](##statistical-disclaimer)

Ofcourse, these data series are not completely random. I wanted them to have:
 1. Interesting narrative potential
 2. Minimize the need for data cleansing
 
Which led me to picking:
- Tesla Production
    - [Available as estimates from Statista](https://www.statista.com/statistics/715421/tesla-quarterly-vehicle-production/)
    - Total number of units sold
- Nonfarm Business Sector: Real Output Per Hour of All Persons
    - [Available from Federal Reserve Bank of St. Louis](https://fred.stlouisfed.org/series/OPHNFB)
    - Taken as an index (100 being 2012 Q3)
***    
## Data
Both time series datasets are exclusively in the range of 2012 Q3 to 2018 Q1.

###   Tesla Production
Given Elon Musk's tendency for secrecy around the inner-workings of Tesla, ([even after a press tour around a section of the manufacturing plant](https://www.cbsnews.com/news/elon-musk-tesla-model-3-problems-interview-today-2018-04-13/)) any specific data for Tesla is hard to come buy. Excluding the publically available financial information.

For this reason my data source is based on estimates calculated from this financial information and fact-checked against Tesla press releases.
    
It represents the *Number of Produced Teslas* of any model since 2012 Q3.

### Real Output per Hour
Based on the measure from "Nonfarm Business Sector: Real Output Per Hour of All Persons", **Nonfarm** is defined by the U.S. Bureau of Labour Statistics (BLS) as:

>... a subset of the domestic economy and excludes the economic activities of the following: general government, private households, nonprofit organizations serving individuals, and farms. 

In general then and in line with the exceptions stated above, it can be taken to be representative of *productivity per hour* in the U.S. economy.

## Statistical Disclaimer
As stated above, this exploratory analysis is by no means complete and still leaves questions unanswered in these datasets. When this task was given to me, I was told to: 
> - select two random data sets
- undertake a correlation comparison and visualisation [on them]
- add some brief documentation
- spend no more than a few minor hours

In light of the last point in particular, I have stopped myself from carrying out further tests in areas that I believe would require following for these insights to "hold water". This would include tests/solutions for:
- The presence of unit roots within the data, which is common in many time series datasets. A good source of information can be found on [Wikipedia](https://en.wikipedia.org/wiki/Unit_root) or [Michael's/Patrick's](https://stats.stackexchange.com/questions/29121/intuitive-explanation-of-unit-root) answers should help for anywone interested in Unit Roots.
- The presence of Heteroskedasticity within the data, shown in the *OLS Residuals* graph at the bottom of this document. This is non-constant variance across a time series, more information at [StatsMakeMeCry.com](http://www.statsmakemecry.com/smmctheblog/confusing-stats-terms-explained-heteroscedasticity-heteroske.html)
- Non-Linear relationship will be fitted poorly by OLS, testing Non-Linear Regression model [alternatives](http://blog.minitab.com/blog/adventures-in-statistics-2/what-is-the-difference-between-linear-and-nonlinear-equations-in-regression-analysis)
- Sum of Squared Errors (RSS) could also be fitting the model poorly even if the data is linear in its dispersion, more information on the OLS [Error Function](http://www.clockbackward.com/2009/06/18/ordinary-least-squares-linear-regression-flaws-problems-and-pitfalls/)
- Presence of Autocorrelation within either datasets or their produced regression model, a simple explanation of which can be [found here](https://www.quora.com/What-is-autocorrelation)

Given the data being utilised and inspite of these statistical shortfalls, there are plausible relationships to be found in the data. These can be found in the [Conclusion](##conclusion) section at the end of this notebook.

## Tools
This notebooks stack is Python 3 and is being run inside of a virtual environment locally; specifically utilising virtualenv and virtualenvwrapper. Because of this there is no virtualenv file in this repo, instead I have created a requirements.txt file that can be used to install all dependancies required to run this notebook locally!

In this EDA I have utilised *Cufflinks*, *Jupyter Notebooks* (no surprise!), *Pandas*, *Plotly*, and *Statsmodels*.

Jupyter Notebooks was utilised to bring this style of interactive presentation, whilst  still allowing use of Markdown and code snippets. Pandas allows easy and powerful data utilisation and Statsmodels (perhaps not surprisingly) allowed for the running of **OLS** (Ordinary Least Squares) regression and neat presentation of standard output statistics. Cufflinks and Poltly are both visualisation packages which make utilisation of multiple other Python visualisation libraries much easier; such as Matplotlib and Seaborn.

In [1]:
# import matplotlib.pyplot as plt
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import statsmodels.api as sm

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## Imports
As described above in [Tools](#tools), these are the modules I have imported for the use throughout the rest of this Jupyter Notebook. Furthermore, there is a requirements.txt file that will allow you to install and run the notebook locally.

I also have defined options within Plotly that allow for the graphs to be saved and manipulated locally for portability.

In [65]:
df = pd.read_csv("TeslaProduction-BusinessProductivity.csv")
df.shape

(23, 3)

## Dataframe and Shape
Initially I import my collected data from **TeslaProduction-BusinessProductivity.csv** into a Pandas dataframe for easy data storage, manipulation and later easy referencing. 

I also defined the structure of thi data, with 23 instances in each dataset, of which there are 3 in total.
***

In [67]:
df.head()

,Date,Tesla Production,Real Output per Hour
0,2012Q3,321,104.503
1,2012Q4,2400,104.059
2,2013Q1,4900,104.304
3,2013Q2,5150,104.147
4,2013Q3,5500,104.570


In [6]:
df.tail()

,Date,Tesla Production,Real Output per Hour
18,2017Q1,25000,107.683
19,2017Q2,22000,108.142
20,2017Q3,26150,108.849
21,2017Q4,29870,108.943
22,2018Q1,29980,109.050


## Head and Tail
Now that the data is held within a dataframe, I can have an initial peruse of the information present within it. Given that I picked out the data myself I have already seen the data, but in case I have forgotten!

From this it's clear the **Date** column is in Quarter format, starting in 2012Q3 and ending in 2018Q1. 
**Tesla Production** and **Real Output per Hour** are both numeric data types as Ratio and Index respecitvely.
***

In [91]:
df.describe()

,Tesla Production,Real Output per Hour
count,23.000000,23.000000
mean,13925.130435,106.461478
std,9136.266914,1.573489
min,321.000000,104.059000
25%,6674.500000,105.108500
50%,11507.000000,106.668000
75%,22100.000000,107.523500
max,29980.000000,109.050000


## Quick Descriptive Statistics
With this simplistic data, only cursory descriptive statistics need to be sought, as they're: 
* Few in number, no need to sample from the collected data
* Scalar in values, not categorical or ordinal

Even though we don't need to sample, it's best to remember that this is still assumed to be a representative sample of the true distributions of each individual dataset.

From this table, we can see that **Tesla Prodcution** handles larger observations than **Real Output per Hour** in all measures. This will become important later when trying to visualize this data, due to the need for multiple axis to plot against.
Inline with this initial observation, **Tesla Production** has a much larger standard deviation and spread when compared to **Real Output per Hour**.
Both series exhibit a positive growth pattern across time, so are likely to be correlated to some degree.

It should be noted that the 50th Percentile (**50%**) marker here is equivalent to the median value of both series respectively.
***

In [88]:
y0 = df['Tesla Production']
y1 = df['Real Output per Hour']

trace0 = go.Box(
    y=y0,
    name = 'Tesla Production',
    marker = dict(
        color = '#1b073a',
        )
    )
trace1 = go.Box(
    y=y1,
    name = 'Real Output per Hour',
    marker = dict(
        color = '#ff0074',
        )
    )
layout = go.Layout(
            title = 'Broken Box Plots'
        )

data = [trace0, trace1]

fig = dict(data=data, layout=layout)

iplot(fig)

## Mistakes were made ...
In my intitial visualization (the box plot above) I assumed that the plots would be created relative to each other and without concern to their specific values.

However, what is clear from the above graph is that this is a feature yet to be brought to Plotly. Instead resulting in a neatly labelled line in-place of what should have been the **Real Output per Hour** plot.

This was included to hopefully warn others of following the same fate and also as a reminder for myself in future Plotly related work!

P.S. if you click on the purple box in the legend, next to "**Tesla Production**", the **Real Output per Hour** line becomes a **Real Output per Hour** box-plot.

P.S.S. potentially found a solution to this problem [**here**](https://plot.ly/python/multiple-axes/) but don't have time yet to try it!

In [103]:
fig0 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='box', 
    showlegend=False,
    colors = ('#ff0074'),
    boxpoints = 'all',
    )
fig1 = df.iplot(
    columns=['Tesla Production'],
    asFigure=True,
    kind='box', 
    secondary_y=['Tesla Production'],
    showlegend=False,
    title='Comparitive Box Plots',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
    boxpoints = 'all',
)

fig1['data'].extend(fig0['data'])

iplot(fig1)

## Mistakes were corrected ...
After my previous shenanigans with box plots, I revisited the plotly documentatin and found how to draw diagrams directly from a Dataframe with ```df.iplot```.
This completes most of the formatting automatically and so only requried me to input the names of the datasets I desired and join them together.

These echo the findings from before within the [Quick Descriptive Statistics](#quick-descriptive-statistics) section. 
It also becomes apparent within these diagrams that **Tesla Production** has a median tending towards the lower values as **Real Output per Hour** median tends toward the higher values within their respective datasets. Due to both datasets having positive sloping characteristics, it can be inferred that they have outliers at the upper-end and lower-end of their distributions respectively.
***

In [131]:
fig0 = df.iplot(
    columns=['Tesla Production','Real Output per Hour'], 
    asFigure=True, 
    kind='histogram', 
    showlegend=True,
    colors = {'Real Output per Hour':'#ff0074',
              'Tesla Production':'#1b073a'},
    title='Distributions',
    yTitle='Frequency',
    histnorm='probability',
    subplots=True,
    shape=(2,1),
    subplot_titles = True,
    bins=6
    )

iplot(fig0)

## Distributions
Above **Tesla Production** and **Real Output per Hour** are shown as distributions on their respective graphs. This re-iterates the previously stated gaps between their *mean* and *median* values.

It also emphasizes the clustering of observations at the lower end of both datasets, which is a characteristic that will need to be accomodated for within later statistical modelling.
***

In [136]:
fig0 = df.iplot(
    columns=['Tesla Production','Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = {'Real Output per Hour':'#ff0074',
              'Tesla Production':'#1b073a'},
    title='Distributions',
    yTitle='Frequency',
    subplots=True,
    shape=(2,1),
    subplot_titles = True,
    x='Date'
    )

iplot(fig0)

## Scatter Graphs
These highlight the periods of flux and calm in both series being at opposite ends of the time line. 

Furthermore, it highlights the larger growth of the **Tesla Production** in comparison to **Real Output per Hour**, sitting at ~9330% and ~4.3% respectively. The datas peaks and troughs also become highlighted at this point.
***

In [133]:
fig2 = df.iplot(
    columns=['Real Output per Hour'], 
    asFigure=True, 
    kind='scatter', 
    showlegend=False,
    colors = ('#ff0074'),
    x='Date',    
    )
fig3 = df.iplot(
    columns=['Tesla Production'],
    asFigure=True,
    kind='scatter', 
    secondary_y=['Tesla Production'],
    title='Comparitive Time Series',
    yTitle='Real Productivity per Hour ($)',
    secondary_y_title = 'Car Units Produced',
    colors = ('#1b073a'),
    x='Date'
    )

fig3['data'].extend(fig2['data'])

iplot(fig3)

## Combined Time Series
Further comparison between the two time series and their similarities.
***

In [256]:
ROpH=sm.add_constant(df['Real Output per Hour'])
model = sm.OLS(df['Tesla Production'],ROpH)
results = model.fit()
df['Fitted Values']=results.fittedvalues
df['Residuals']=results.resid
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       Tesla Production   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.828
Method:                 Least Squares   F-statistic:                     106.6
Date:                Mon, 25 Jun 2018   Prob (F-statistic):           1.10e-09
Time:                        19:53:04   Log-Likelihood:                -221.13
No. Observations:                  23   AIC:                             446.3
Df Residuals:                      21   BIC:                             448.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -5.511e+05 

## OLS Regression
Above I utilise Ordinary Least Squares (**OLS**), a classic linear regression technique to evaluate the correlation between **Tesla Production** and **Real Output per Hour** and provide its standard statistical output. This is utilising the ```statsmodels``` package, which includes many other useful statistical tools and models.

I chose OLS as it is, simple, fast and easy to interpret with regards to its output. Given the parameters of my task which I defined in my [Statistical Disclaimer](#statistical-disclaimer) I believed OLS to be a solid choice for cursory Exploratory Data Analysis. OLS does come with some caveats which I have discussed in my [Statistical Disclaimer](#statistical-disclaimer) as it tries to obey the assumptions of the **Classic Linear Regression Model**.

Given the small number of observations (23) this model will be susceptible to bias in both our observed data and the residuals produced by it; since it won't be representative of the true population it's attempting to estimate.
Inspite of this, the R-squared and adjusted R-squared value is 83.5% and 82.8% respectively, with the latter accounting for the number of dependant variables.
The other associatted values around there of F-statistic, Prob(F-statistic), Log-Likelihood, AIC and BIC would be useful for comparing the fits of different model specifications between each other. But due to only running a single test, will be useless in this example.

The **const** represents the point at which the regression line crosses the y-axis and helps to collect some of the noise from a regression. It has few practical applications for inference and is predominately included to improve model performance, which is always good to work on!

The coefficient of 5307.3116 for **Tesla Production** shows that for a every 1 unit increase in **Real Output per Hour**, there would be an extra 5307.3116 Tesla Vehicles produced. This is stated with a standard error of 512.923 is one standard deviation away from the estimate of the coefficient and a t-statistic of 10.071. This value with 22 Degrees of Freedom results in the coefficient being statistically significant for > p = 0.001, which is of a confidence interval greater than 0.1%.

The Skew value of -0.579 shows that the regressions peak tends towards infinity, which is intuitive in this example given the previously discussed positive growth and fluctutations. Kurtosis of 2.385 shows how much of the variance is the result of infrequent extreme variations, but has not upper limit and so cannot be inferred from directly.
So similar to Skew, Kurtosis is best used when compared relatively to other datasets or models.

Finally, the Durbin-Watson statistic is a measure of autocorrelation from the residuals of a regression. It is measured between 0 and 4, where 2 would represent no autocorrelation, 0 and 4 representing autocorrelation in their positive and negative slopes respectively. Autocorrelation was previously discussed as a potential problem in my [Statistical Disclaimer](#statistical-disclaimer) and has turned out to be true. With a value of 1.034, the resultant model has strong positive autocorrelation which will have invalidated the previious findings I discussed.
This can be accomodated for by utilising more complex models such as **ARMA**. A **ARIMA** model may be more pertinent though as this would also help to elimate the issues with heteroskedasticity and possible unit roots.

The last two commands in the output save the models **'Fitted Values'** and **'Residuals'** into my pandas dataframe with their respective column names.
***

In [257]:
fig0 = df.iplot(
    columns=['Tesla Production', 'Real Output per Hour', 'Fitted Values', 'Residuals'], 
    asFigure=True, 
    kind='scatter', 
    colors = {'Real Output per Hour':'#ff0074',
          'Tesla Production':'#1b073a',
          'Fitted Values': 'black',
          'Residuals': 'red',
         },
    x='Date',
    title='OLS Fitted Values Comparison',
    subplots= True,
    shape = (2,2),
    subplot_titles = True,
    )

iplot(fig0)

## OLS Fitted Values - Scatter Graph
These are a collection of graphs representing the input and output data of OLS.

In the bottom left graph are plotted the **Fitted Values** from the OLS regression. This represents the data input to the model and the relationship they both share, from the OLS perspective.

In the bottom right graph are the plotted **Residuals** from the OLS regression. These can be seen to *generally* orbiting around the origin, so to be "stationary". However, there does seem to be an oscillating trend over the range of the data and so would still need to be investigated thoroughly.
These **Residuals** would also indicate that the model on the whole is not of a good fit, due to an error in places being *>7,000*. Again, this further adds weight to utilising non-parametric models to allow for this non-linear relationship.
***

In [258]:
fig0 = df.iplot(
    columns=['Tesla Production','Real Output per Hour', 'Fitted Values', 'Residuals'], 
    asFigure=True, 
    kind='histogram', 
    showlegend=True,
    colors = {'Real Output per Hour':'#ff0074',
              'Tesla Production':'#1b073a',
              'Fitted Values': 'black',
              'Residuals': 'red',
             },
    title='Distributions',
    yTitle='Frequency',
    histnorm='probability',
    subplots=True,
    shape=(2,2),
    subplot_titles = True,
    bins=6,
#     dimensions=(1000,1200)
    )

iplot(fig0)

## OLS Fitted Values - Histogram
This shows the distribution of the **Fitted Values**, which since "fitting" in OLS have become more normally distributed. There is still a clustering around the origin which would could be accomodated for through models such as Weighted Least Squares.

The **Residuals** show a near-normal distribution on the negative side of the origin, but a bias on the positive suggesting that the model is consistently over-estimating the observed data and when it under-estimates does so by a smaller margin of error.
***

In [ ]:
Auto correlation of errors, they are related, ommitted variables etc.

In [259]:
df['Tesla Production'].corr(df['Real Output per Hour'], method='spearman')

0.9456521739130436

## Spearman Rank Correlation
This measures the monotonic relationship between two datasets, such that as one variable gets larger so does the other. It has been established throughout this notebook that there is a correlation between these two datasets, but we are yet to test for a relationship that is non-linear in nature.

Spearmans Rank allows for us to quickly and succintly measure the correlation of any non-parametric interval/ratio/ordinal data with easy interpretation. The returned value ranges from -1 to +1, where ±1 represents a perfect relationship between the data and 0 represents no relationship. Positive or negative signs on the value represents the positive/negative style to the relationship.

In this case Spearman Rank = 0.946 (3 d.p.) which shows an extremely strong monotonic relationship between the data. Spearmans value is noticeably higher than the adj. R-squared = 0.708, primarily due to the latter relying on a linear relationship between the datasets. Adding weight to the arguement from earlier in this notebook that a non-parametric (non-linear) model could fit the relationship better.
***

## Conclusion
Despite the statistical flaws outligned in the [Statistical Disclaimer](##statistical-disclaimer), there plausible insights that have been found. I've also tried to point out throughout the workbook what direction to go in to make these results robust; even though they are likely to always be spurious given the nature of the datasets!

The inference that Tesla Production, if the figures could be known directly from Tesla, could be a leading indicator of Real Output per Hour is plausible. Since the latter is released on the first Friday of the following month by the Bureau of Labour Statistics, there will usually be a lead on the Tesla data by a couple of days. Resulting in it being a potential predictor of USD($) Forex price estimation, given its large effect on the [financial markets](https://www.poundsterlinglive.com/usd/8831-us-dollar-eyes-non-farm-payrolls-and-wages-data-as-latest-trade-tensions-rile-markets).

For future work purposes, these hurdles would need to be overcome with these specific pieces of data:
- Non-parametric model would fit better
- Heteroskedasticity robust 
- Potentially unit-root robust



